In [7]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [7]:
import mysql.connector
import pandas as pd

host = "localhost"
user = "root"
password = "vachas289"
database_name = "VrindaStoreData"
file_path = r"C:\Users\al6436\Desktop\Vrinda Store Data Analysis.xlsx"

def create_mysql_connection():
    return mysql.connector.connect(
        host=host,
        user=user,
        password=password
    )

def create_schema(cursor):
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    cursor.execute(f"USE {database_name}")

def clean_column_names(columns):
    cleaned_columns = []
    for col in columns:
        cleaned_col = col.strip().replace(" ", "_").replace("-", "_")
        cleaned_columns.append(cleaned_col)
    return cleaned_columns

def read_excel(file_path):
    df = pd.read_excel(file_path, engine='openpyxl')
    
    df.columns = clean_column_names(df.columns)
    
    return df

def create_table_from_df(cursor, df):
    table_name = "VrindaStoreDataTable"
    
    columns = ", ".join([f"`{col}` VARCHAR(255)" for col in df.columns])
    
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (id INT AUTO_INCREMENT PRIMARY KEY,{columns})
    """
    
    cursor.execute(create_table_query)

def insert_data(cursor, df):
    table_name = "VrindaStoreDataTable"
    
    for _, row in df.iterrows():
        columns = ", ".join([f"`{col}`" for col in df.columns])
        values = ", ".join([f"'{str(value)}'" for value in row])
        
        insert_query = f"""
        INSERT INTO {table_name} ({columns})
        VALUES ({values})
        """
        
        cursor.execute(insert_query)

def main():
    try:
        conn = create_mysql_connection()
        cursor = conn.cursor()
        
        create_schema(cursor)
        
        df = read_excel(file_path)
        
        create_table_from_df(cursor, df)
        
        insert_data(cursor, df)
        
        conn.commit()
        
        print(f"Data from {file_path} has been successfully uploaded to MySQL.")
    
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    main()


Data from C:\Users\al6436\Desktop\Vrinda Store Data Analysis.xlsx has been successfully uploaded to MySQL.
